# Full Photometry Pipeline

This tutorial demonstrates how to run the end--to--end photometry pipeline on
real JWST data included with *Mophongo*. We build templates from the F444W
mosaic and fit fluxes in the F770W image using spatially varying PSF
matching kernels.

## Detect sources in F444W

In [ ]:
from pathlib import Path
import numpy as np
from astropy.io import fits

data_dir = Path('../data')
field = 'uds-test'
#field = 'uds-medium'
#field = 'uds-large'
sci_444 = data_dir / f'{field}-f444w_sci.fits'
wht_444 = data_dir / f'{field}-f444w_wht.fits'


## Build PSF region map

In [ ]:
from mophongo.psf import DrizzlePSF
from mophongo.psf_map import PSFRegionMap

# initialize drizzler; also reads the associated _wcs.csv files (rate files wcs/header information)
dpsf_444 = DrizzlePSF(driz_image=str(sci_444))
dpsf_770 = DrizzlePSF(driz_image=str(sci_444).replace('444', '770'))

# map unique detector overlaps; keep only footprints overlapping the target mosaic
prm_444 = PSFRegionMap.from_footprints(dpsf_444.footprint, name='F444W').overlay_with(dpsf_444.driz_footprint)
prm_770 = PSFRegionMap.from_footprints(dpsf_770.footprint, name='F770W').overlay_with(dpsf_444.driz_footprint)

# compute overlay regions unique to both PSFs
prm_kern = prm_444.overlay_with(prm_770)
prm_444.plot()
prm_770.plot()
prm_kern.plot()
prm_kern.regions

## Create PSF kernels

In [ ]:
import mophongo.utils as utils

psf_dir = data_dir / 'PSF'
stpsf_444 = 'UDS_NRC.._F444W_OS4_GRID25'
stpsf_770 = 'UDS_MIRI_F770W_OS4_GRID9'
size = 101

# centroid positions of the regions
pos = [np.squeeze(p.xy) for p in prm_kern.regions.geometry.centroid]

# load webb psfs
dpsf_444.epsf_obj.load_jwst_stdpsf(local_dir=str(psf_dir), filter_pattern=stpsf_444)
dpsf_770.epsf_obj.load_jwst_stdpsf(local_dir=str(psf_dir), filter_pattern=stpsf_770)

# drizzle at centroid positions
prm_444.psfs = dpsf_444.get_psf_radec(pos, size=size)
prm_770.psfs = dpsf_770.get_psf_radec(pos, size=size)

# store the PSFs + region maps
prm_444.to_file(data_dir / f'{field}-f444w_psf.geojson')
prm_770.to_file(data_dir / f'{field}-f770w_psf.geojson')

# match kernels
# @@@ need a better way to determine best fft window shape
kernels = [utils.matching_kernel(psf_444, psf_770, recenter=True) for psf_444, psf_770 in zip(prm_444.psfs, prm_770.psfs)]

prm_kern.psfs = np.asarray(kernels)
prm_kern.to_file(data_dir / f'{field}-f444w_kernel_f770w.geojson')

## Run photometry

In [ ]:
from pathlib import Path
from astropy.io import fits
from mophongo.psf_map import PSFRegionMap
from mophongo.fit import FitConfig
from mophongo.catalog import Catalog
from mophongo import pipeline

data_dir = Path('../data')
field = 'uds-test'
field = 'uds-medium'
#field = 'uds-large'
sci_444 = data_dir / f'{field}-f444w_sci.fits'
wht_444 = data_dir / f'{field}-f444w_wht.fits'

prm_kern = PSFRegionMap.from_geojson(str(data_dir / f'{field}-f444w_kernel_f770w.geojson'))
prm_444 = PSFRegionMap.from_geojson(str(data_dir / f'{field}-f444w_psf.geojson'))
prm_770 = PSFRegionMap.from_geojson(str(data_dir / f'{field}-f770w_psf.geojson'))

# MIRI images
img_770 = fits.getdata(data_dir / f'{field}-f770w_sci.fits')
wht_770 = fits.getdata(data_dir / f'{field}-f770w_wht.fits')

# read in LW selected catalog
seg_LW = data_dir / f'{field}-LW_seg.fits'
cat444 = Catalog.from_fits(sci_444, wht_444, segmap=seg_LW, estimate_ivar=True)
cat770 = Catalog.from_fits(img_770, wht_770, segmap=seg_LW, estimate_ivar=True) 

residual = [None, None, None, None]


In [ ]:
# first fit, no shifts: first image is template, 2nd and on the fitting images
table, res, fit = pipeline.run([cat444.sci, cat770.sci], cat444.segmap.data,
                                weights=[cat444.ivar, cat770.ivar],
                                catalog=cat444.table,
                                psfs=[prm_444, prm_770],
                                kernels=[None, prm_kern],
                                wcs=[cat444.wcs]* 2,
                                config=FitConfig(fit_astrometry_niter=0))
residual[0] = res[0]
fits.writeto('residual_noshift.fits', data=residual[0], overwrite=True)
fits.writeto('model_noshift.fits', data=cat770.sci-residual[0], overwrite=True)

### Separate flux and astrometric fit

First fit fluxes, then astrometric shift field, iterate.

In [ ]:
# test astrometry fit vs number of iterations for separate astro + flux fits
config = FitConfig(fit_astrometry_niter=2,
                    fit_astrometry_joint=False,
                    astrom_basis_order=2,
                    reg_astrom=1e-4,
                    snr_thresh_astrom=10.0,
                    multi_tmpl_chi2_thresh=3.0)
table1, res1, fit1 = pipeline.run([cat444.sci, cat770.sci],
                                cat444.segmap.data,
                                weights=[cat444.ivar, cat770.ivar],
                                catalog=cat444.table,  
                                psfs=[prm_444, prm_770],
                                kernels=[None, prm_kern],
                                wcs=[cat444.wcs] * 2,
                                config=config)
residual[1] = res1[0]
fits.writeto('residual_separate.fits', data=residual[1], overwrite=True)
fits.writeto('model_separate.fits', data=cat770.sci-residual[1], overwrite=True)

###  Joint photometric and astrometric fit

Append Cheby polynomial coefficients to sparse matrix and solve jointly.

In [ ]:
# test astrometry fit vs number of iterations for joint astro + flux fit
config = FitConfig(fit_astrometry_niter=3,
                    fit_astrometry_joint=True,
                    astrom_basis_order=2,
                    reg_astrom=1e-4,
                    snr_thresh_astrom=10.0)
table2, res2, fit2 = pipeline.run([cat444.sci, cat770.sci],
                                cat444.segmap.data,
                                weights=[cat444.ivar, cat770.ivar],
                                catalog=cat444.table,
                                psfs=[prm_444, prm_770],
                                kernels=[None, prm_kern],
                                wcs=[cat444.wcs] * 2,
                                config=config)
residual[2] = res2[0]
fits.writeto('residual_joint.fits', data=residual[2], overwrite=True)
fits.writeto('model_joint.fits', data=cat770.sci-residual[2], overwrite=True)

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(2, 2, figsize=(15, 12))
ax = ax.flatten()
titles= ['No Shift', 'Separate Astro + Flux', 'Joint Astro + Flux']
ax[0].imshow(cat770.sci, origin='lower', cmap='gray', vmin=-3, vmax=3)
for i in range(3):
    ax[i+1].imshow(residual[i], origin='lower', cmap='gray', vmin=-1, vmax=1)
    ax[i+1].set_title(titles[i])
plt.tight_layout()

In [ ]:
from matplotlib import pyplot as plt
img = [cat444.sci*10, cat770.sci, cat770.sci - residual[2], residual[2]] 
titles = ['F444W', 'F770W', 'model F770W', 'residual']
fig, ax = plt.subplots(2, 2, figsize=(15, 12))
ax = ax.flatten()
for i in range(4):
    ax[i].imshow(img[i], origin='lower', cmap='gray', vmin=-1, vmax=1)
    ax[i].set_title(f'{titles[i]}')
plt.tight_layout() 